In [1]:
#import ydata_profiling

SEED = 2022
#!/usr/bin/env python
import sys
import os
sys.path.append('src/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA
import logging
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import (
    TimeSeriesSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
import time
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
from xgboost import XGBRegressor

# Load the data

In [2]:
train_df = pd.read_csv('Data/housing_price/train.csv', index_col=0)

In [3]:
train_df.reset_index(inplace=True, drop=True)

In [4]:
train_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
drop_features = ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [6]:
train_df.drop(columns=drop_features, inplace=True)

In [7]:
def fit_processor(X_train, numeric_features, categorical_features):
    """
    Applies Simple Imputer to Categorical Features
    Applies One Hot Encoding to Categorical Features
    Applies Quantile Scaling to Numeric Features
    Returns and writes pickle file of the complete preprocessor

    Parameters
    ----------
    X_train : numpy
        Training Data in NumPy format
    numeric_features : list[string]
        List of Numeric Features
    categorical_features: list[string]
        List of Categorical Features

    Returns
    -------
    preprocessor : sklearn.Preprocessor
        sklearn preprocessor fit on the training set
    """
    pipe_num = Pipeline([
        ('impute', SimpleImputer(strategy='mean')),
        ('scaler',  MinMaxScaler())
    ])
    pipe_cat = Pipeline([
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]) 
    preprocessor = ColumnTransformer([
        ('num', pipe_num, numeric_features),
        ('cat', pipe_cat, categorical_features)
    ])

    norm_X_train = preprocessor.fit(X_train)

    return preprocessor

In [8]:
def format_data(df):
    """
    Format the Data into X, y to setup different variables for the Machine Learning pipeline

    Parameters
    ----------
    df : DataFrame
        DataFrame to separate into X,y

    Returns
    -------
    Xy : tuple[numpy, numpy]
        NumPy arrays of features and predictor

    """
    y = df['SalePrice']
    y = y.to_numpy()
    col_names = df.columns.tolist()
    col_names.remove('SalePrice')
    X = df[col_names]

    return X,y

# CHKN NOTES
- train validation and test - 3 fundos
- split it 80 20
- crossvalidation while trainng
- tune hyper paras of model
- but method to it are diff across each thing
- best one to use for tabular is called stratified k fold - time is time- why bother????
- CROSSValidate

In [10]:
from sklearn.model_selection import train_test_split

In [12]:
features = train_df.columns.to_list()

In [13]:
X = train_df[features]
y = train_df[['SalePrice']]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [15]:
# Prepare columns
features = X_train.columns.to_list()
df_numerical_features = X_train.select_dtypes(include='number')
df_categorical_features = X_train.select_dtypes(include='object')

# preprocess
features = X_train.columns.to_list()
numerical_features = df_numerical_features.columns.to_list()
categorical_features = df_categorical_features.columns.to_list()
processor = fit_processor(X_train[features],numerical_features, categorical_features)

In [23]:
norm_X_train = processor.transform(X_train)

In [16]:
X_test.shape

(292, 75)

In [17]:
y_train.shape

(1168, 1)

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Chknnotes2
1. Decide the features
2. Train test split data into X, y
3. Decide the preprocessing
4. Implement crossvalidation
5. test

In [19]:
params = {
    'n_estimators':[10,50,100],
    'max_depth':[50,100,200],
    'min_samples_split':[2,3,4]    
}

In [20]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
skf = KFold(n_splits=10, random_state=69, shuffle=True)

In [21]:
clf = GridSearchCV(RandomForestRegressor(), params, cv=skf)

In [ ]:
clf.fit(norm_X_train, y_train.values.ravel())

In [ ]:
clf.best_params_

In [ ]:
clf.score(norm_X_train, y_train)

In [ ]:
clf.score(norm_X_test, y_test)

In [ ]:
test_df = pd.read_csv('Data/housing_price/test.csv', index_col=0)

In [ ]:
test_df.reset_index(inplace=True, drop=True)

In [ ]:
test_df.tail()

In [ ]:
test_df = test_df[features]

In [ ]:
test_df

In [ ]:
test_df = test_df.fillna(0)

In [ ]:
norm_X_test_final = scaler.transform(test_df)


In [ ]:
norm_X_test_final

In [ ]:
y_predict = clf.predict(norm_X_test_final)

In [ ]:
y_final = pd.DataFrame(y_predict, columns=['SalePrice'])

In [ ]:
final_df = pd.concat([test_df, y_final], axis=1)

In [ ]:
final_df.tail()

In [ ]:
import shap
shap.initjs()

In [ ]:
explainer = shap.Explainer(clf.best_estimator_)
shap_values = explainer(norm_X_test)

In [ ]:
shap.summary_plot(shap_values, norm_X_test, feature_names=features)